In [5]:
# ====================================================================================
# 0. 기본 설정 및 라이브러리 임포트
# ====================================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- 기본 설정 ---
# 학습에 사용할 문자셋. 실제 데이터셋에 맞게 수정해야 합니다.
GT_FILE_PATH = r"DBNet_OCR/data/crop/gt.txt" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def generate_character_set(gt_file):
    if not os.path.exists(gt_file):
        print(f"오류: gt.txt 파일을 찾을 수 없습니다! 경로를 확인하세요: {gt_file}")
        return None # 오류 발생 시 None 반환

    all_characters = set()
    with open(gt_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                _, text = line.strip().split('\t', 1)
                for char in text:
                    all_characters.add(char)
            except ValueError:
                continue

    sorted_characters = sorted(list(all_characters))
    final_charset = "".join(sorted_characters)
    
    print("="*50)
    print("CHARACTER_SET 생성이 완료되었습니다.")
    print(f"총 글자 수: {len(final_charset)}")
    print("="*50)
    
    return final_charset # 생성된 문자열을 반환


# ====================================================================================
# 1. 레이블 변환기 (CTCLabelConverter)
# - 역할: 텍스트 문자열을 모델이 이해할 수 있는 숫자 시퀀스로, 
#         모델의 출력(숫자 시퀀스)을 다시 텍스트 문자열로 변환합니다.
# ====================================================================================
class CTCLabelConverter:
    """텍스트와 인덱스 간의 변환을 담당하는 클래스"""
    
    def __init__(self, character_set):
        # 0번 인덱스는 CTC Loss를 위한 'blank' 토큰으로 예약합니다.
        self.character_set = ["-"] + list(character_set)
        
        # 문자를 인덱스로 변환하는 딕셔너리
        self.char_to_idx = {char: i for i, char in enumerate(self.character_set)}
        # 인덱스를 문자로 변환하는 딕셔너리
        self.idx_to_char = {i: char for i, char in enumerate(self.character_set)}
        
        print(f"인식 모델의 클래스 개수 (blank 포함): {self.get_num_classes()}")

    def encode(self, text):
        """입력된 텍스트 문자열을 숫자 인덱스의 리스트로 변환합니다."""
        # character_set에 없는 문자는 무시합니다.
        indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices):
        """모델의 출력(인덱스 시퀀스)을 텍스트 문자열로 디코딩합니다."""
        # CTC Greedy Decode 방식:
        # 1. 가장 확률이 높은 인덱스를 선택합니다.
        # 2. 연속되는 중복 인덱스를 제거합니다.
        # 3. blank(인덱스 0) 토큰을 제거합니다.
        text = []
        last_idx = 0
        for idx in indices:
            idx_item = idx.item()
            if idx_item == 0:  # blank 토큰은 무시
                last_idx = 0
                continue
            if idx_item == last_idx:  # 연속 중복 문자 무시
                continue
            
            text.append(self.idx_to_char[idx_item])
            last_idx = idx_item
            
        return "".join(text)

    def get_num_classes(self):
        """blank 토큰을 포함한 전체 클래스의 개수를 반환합니다."""
        return len(self.character_set)

# ====================================================================================
# 2. 데이터 파이프라인 (Dataset 및 DataLoader)
# - 역할: 디스크에 저장된 이미지와 텍스트 파일을 불러와 모델 학습에 사용할 수 있는
#         PyTorch 텐서(Tensor) 형태로 변환하고, 배치(batch) 단위로 묶어줍니다.
# ====================================================================================

class RecognitionDataset(Dataset):
    """인식용 데이터셋을 위한 클래스"""
    def __init__(self, gt_file_path, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.samples = []
        # gt.txt 파일을 읽어 (이미지 파일명, 텍스트) 쌍을 리스트에 저장합니다.
        with open(gt_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    filename, text = line.strip().split('\t')
                    self.samples.append((filename, text))
                except ValueError:
                    # 탭으로 분리되지 않은 줄은 건너뜁니다.
                    print(f"경고: 잘못된 형식의 라인 발견 - {line.strip()}")
                    continue

    def __len__(self):
        # 전체 데이터셋의 샘플 수를 반환합니다.
        return len(self.samples)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 샘플을 반환합니다.
        filename, text = self.samples[idx]
        image_path = os.path.join(self.image_dir, filename)
        
        # OpenCV로 이미지를 읽습니다.
        image = cv2.imread(image_path)
        # BGR을 RGB로 변환합니다.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 데이터 증강 및 텐서 변환을 적용합니다.
        if self.transform:
            image = self.transform(image=image)['image']
            
        return image, text

def get_recognition_transforms(height, width):
    """인식 모델 학습을 위한 데이터 증강 파이프라인을 정의합니다."""
    return A.Compose([
        # CRNN은 입력 이미지의 높이가 고정되어야 하므로, 항상 리사이즈합니다.
        A.Resize(height, width, always_apply=True),
        # 밝기, 대비 등 색상 관련 증강을 적용합니다.
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
        # 이미지를 -1에서 1 범위로 정규화합니다.
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        # PyTorch 텐서로 변환합니다.
        ToTensorV2(),
    ])

def recognition_collate_fn(batch, converter):
    """
    가변 길이의 텍스트 레이블을 처리하기 위한 커스텀 collate 함수.
    DataLoader가 이 함수를 사용해 여러 샘플을 하나의 배치로 만듭니다.
    """
    images, texts = zip(*batch)
    # 이미지들은 크기가 같으므로 stack을 이용해 하나의 텐서로 합칩니다.
    images = torch.stack(images, 0)
    
    # 텍스트들을 인코딩합니다.
    encoded_texts = [converter.encode(text) for text in texts]
    # 각 텍스트의 길이를 저장합니다.
    target_lengths = torch.tensor([len(t) for t in encoded_texts], dtype=torch.long)
    # 모든 텍스트의 인덱스를 하나의 1D 텐서로 이어붙입니다.
    targets = torch.cat(encoded_texts)
    
    return images, targets, target_lengths


# ====================================================================================
# 3. 텍스트 인식 모델 (CRNN) 아키텍처
# - 역할: (CNN) 이미지에서 특징 추출 -> (RNN) 특징의 순서와 문맥 학습 -> (Classifier) 글자 예측
# ====================================================================================

class CRNN(nn.Module):
    def __init__(self, num_chars, rnn_hidden_size=256, rnn_layers=2):
        super().__init__()
        
        # --- 1. CNN 특징 추출기 (VGG 스타일) ---
        # 입력 이미지: (B, 3, 32, 100)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 64, 16, 50)
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 128, 8, 25)
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 256, 4, 25)
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 512, 2, 25)
            nn.Conv2d(512, 512, (2,1), 1, 0), nn.BatchNorm2d(512), nn.ReLU(True)  # -> (B, 512, 1, 25)
        )
        
        # --- 2. RNN (LSTM) 문맥 학습기 ---
        # CNN 출력 특징맵을 RNN이 처리할 수 있는 시퀀스 형태로 변환
        self.rnn = nn.LSTM(
            input_size=512,          # CNN 출력의 채널 수
            hidden_size=rnn_hidden_size,
            num_layers=rnn_layers,
            bidirectional=True,      # 양방향 RNN으로 더 넓은 문맥 파악
            dropout=0.5
        )
        
        # --- 3. Classifier (분류기) ---
        # RNN의 출력을 각 문자의 확률로 변환
        self.classifier = nn.Linear(rnn_hidden_size * 2, num_chars) # 양방향이므로 *2

    def forward(self, x):
        # 1. CNN을 통과시켜 이미지 특징 추출
        features = self.cnn(x)  # -> (Batch, Channels, Height, Width) = (B, 512, 1, 25)
        
        # 2. RNN 입력 형식으로 변환: (SeqLen, Batch, InputSize)
        b, c, h, w = features.size()
        assert h == 1, "CNN 출력의 높이는 1이어야 합니다."
        features = features.squeeze(2)      # 높이(H) 차원 제거 -> (B, 512, 25)
        features = features.permute(2, 0, 1)  # 차원 순서 변경 -> (W, B, C) = (25, B, 512)
        
        # 3. RNN을 통과시켜 문맥 정보 학습
        rnn_output, _ = self.rnn(features) # -> (SeqLen, Batch, HiddenSize*2)
        
        # 4. 각 시퀀스 스텝에 대해 문자 분류
        output = self.classifier(rnn_output) # -> (SeqLen, Batch, NumClasses)
        return output

# ====================================================================================
# 4. 모델 학습 스크립트
# ====================================================================================
def train_recognizer(character_set, gt_file_path, image_dir):
    print("===== 텍스트 인식 모델 학습 시작 =====")
    
    # --- 2. 설정 ---
    IMG_HEIGHT = 32
    IMG_WIDTH = 100
    BATCH_SIZE = 32
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.1
    
    # --- 3. 데이터 파이프라인 준비 ---
    print("데이터 로더를 준비합니다...")
    converter = CTCLabelConverter(character_set)
    NUM_CLASSES = converter.get_num_classes()
    
    dataset = RecognitionDataset(
        gt_file_path=gt_file_path,
        image_dir=image_dir,
        transform=get_recognition_transforms(IMG_HEIGHT, IMG_WIDTH)
    )
    
    if len(dataset) == 0:
        print("오류: 데이터셋에 샘플이 없습니다. 경로를 확인해주세요.")
        return None, None

    data_loader = DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0, # Windows 환경에서는 0이 안정적
        collate_fn=lambda batch: recognition_collate_fn(batch, converter)
    )

    # --- 4. 모델, 손실함수, 옵티마이저 준비 ---
    print("모델과 옵티마이저를 준비합니다...")
    model = CRNN(num_chars=NUM_CLASSES).to(DEVICE)
    criterion = nn.CTCLoss(blank=0, zero_infinity=True, reduction='mean')
    optimizer = torch.optim.Adadelta(model.parameters(), lr=LEARNING_RATE)

    # --- 5. 학습 루프 ---
    for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = 0
        progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        for images, targets, target_lengths in progress_bar:
            images, targets, target_lengths = images.to(DEVICE), targets.to(DEVICE), target_lengths.to(DEVICE)
            
            preds = model(images)
            log_probs = F.log_softmax(preds, dim=2)
            input_lengths = torch.full(size=(images.size(0),), fill_value=preds.size(0), dtype=torch.long)
            
            loss = criterion(log_probs, targets, input_lengths, target_lengths)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

        avg_loss = epoch_loss / len(data_loader)
        print(f"\nEpoch {epoch+1} 완료, 평균 손실: {avg_loss:.4f}")

        # --- 6. 간단한 검증 ---
        model.eval()
        with torch.no_grad():
            try:
                # next(iter())는 데이터로더에서 딱 한 배치만 가져옵니다.
                val_images, val_texts, _ = next(iter(data_loader)) # target_lengths는 사용하지 않으므로 _로 받음
                val_preds = model(val_images.to(DEVICE)).argmax(2).permute(1, 0)
                
                print("--- 검증 예시 ---")
                for i in range(min(4, len(val_texts))):
                    decoded_text = converter.decode(val_preds[i])
                    print(f"  GT: '{val_texts[i]:<20}' | PRED: '{decoded_text}'")
            except StopIteration:
                print("검증을 위한 데이터가 부족합니다.")

    # --- 7. 학습된 모델 저장 ---
    torch.save(model.state_dict(), "crnn_recognizer_final.pth")
    print("\n학습 완료! 모델이 'crnn_recognizer_final.pth'로 저장되었습니다.")
    return "crnn_recognizer_final.pth", converter

# ====================================================================================
# 5. 학습된 모델을 사용한 추론(Inference) 모듈
# - 역할: 학습이 끝난 모델 가중치(.pth)를 불러와 새로운 이미지의 글자를 읽습니다.
# ====================================================================================
# ====================================================================================
# 메인 실행 블록 (최종 수정본)
# ====================================================================================
if __name__ == '__main__':
    # --- [1단계: 설정] 학습할 데이터의 경로를 여기에 정의합니다. ---
    GT_FILE_PATH = r"C:\Users\User\DBNet_OCR\data\crop\gt.txt" # 본인의 gt.txt 경로
    IMAGE_DIR = r"C:\Users\User\DBNet_OCR\data\crop\images"   # 본인의 images 폴더 경로

    # --- [2단계: 문자셋 생성] 위 경로의 gt.txt 파일로부터 학습에 필요한 모든 글자를 추출합니다. ---
    print(">> 1. 문자셋(CHARACTER_SET)을 생성합니다...")
    charset = generate_character_set(GT_FILE_PATH)
    
    # --- [3단계: 학습 시작] 문자셋이 성공적으로 생성되면, 이 정보들을 가지고 학습을 시작합니다. ---
    if charset:
        print("\n>> 2. 모델 학습을 시작합니다...")
        # train_recognizer 함수에 필요한 모든 정보(charset, gt_file_path, image_dir)를 인자로 전달합니다.
        trained_model_path, label_converter = train_recognizer(charset, GT_FILE_PATH, IMAGE_DIR)
        
        # --- [4단계: 추론 테스트] 학습이 성공적으로 끝나면, 결과 모델로 테스트를 진행합니다. ---
        if trained_model_path and label_converter:
            print("\n>> 3. 추론 테스트를 시작합니다...")
            recognizer = Recognizer(trained_model_path, label_converter, DEVICE)
            
            try:
                # 테스트할 이미지 선택 (학습 데이터셋의 첫 번째 이미지로 테스트)
                test_image_name = os.listdir(IMAGE_DIR)[0]
                test_image_path = os.path.join(IMAGE_DIR, test_image_name)
                
                # 예측 실행
                predicted_text = recognizer.predict(test_image_path)
                
                print(f"\n--- 테스트 결과 ---")
                print(f"이미지: {test_image_path}")
                print(f"예측된 텍스트: '{predicted_text}'")
            except (IndexError, FileNotFoundError):
                print("테스트할 이미지를 찾을 수 없습니다.")
    else:
        print("문자셋 생성에 실패하여 학습을 시작할 수 없습니다.")

>> 1. 문자셋(CHARACTER_SET)을 생성합니다...
CHARACTER_SET 생성이 완료되었습니다.
총 글자 수: 1550

>> 2. 모델 학습을 시작합니다...
===== 텍스트 인식 모델 학습 시작 =====
데이터 로더를 준비합니다...
인식 모델의 클래스 개수 (blank 포함): 1551
경고: 잘못된 형식의 라인 발견 - rec_crop_00002547.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00007143.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00012858.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00044170.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00097253.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00123738.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00127828.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00208717.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00249756.png


C:\Users\User\AppData\Local\Temp\ipykernel_4144\3080662519.py:144: UserWarning: Argument(s) 'always_apply' are not valid for transform Resize
  A.Resize(height, width, always_apply=True),


경고: 잘못된 형식의 라인 발견 - rec_crop_00329351.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00345028.png
모델과 옵티마이저를 준비합니다...


Epoch 1/50: 100%|███████████████████████████████████████████████████| 12175/12175 [53:07<00:00,  3.82it/s, loss=0.8309]



Epoch 1 완료, 평균 손실: 3.7081
--- 검증 예시 ---
  GT: '37                  ' | PRED: 'o'
  GT: '40                  ' | PRED: 'R'
  GT: '327                 ' | PRED: '비는의'
  GT: '748                 ' | PRED: 'G'


...


Epoch 49 완료, 평균 손실: 0.0603
--- 검증 예시 ---
  GT: '133                 ' | PRED: '거'
  GT: '169                 ' | PRED: '과'
  GT: '1365                ' | PRED: '트'
  GT: '1379                ' | PRED: '판'


Epoch 50/50: 100%|██████████████████████████████████████████████████| 12175/12175 [13:21<00:00, 15.20it/s, loss=0.0076]
C:\Users\User\AppData\Local\Temp\ipykernel_4144\128659728.py:322: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on Git


Epoch 50 완료, 평균 손실: 0.0576
--- 검증 예시 ---
  GT: '436                 ' | PRED: '들'
  GT: '63                  ' | PRED: 'lonely'
  GT: '66                  ' | PRED: '하'
  GT: '65                  ' | PRED: '영'

학습 완료! 모델이 'crnn_recognizer_final.pth'로 저장되었습니다.

>> 3. 추론 테스트를 시작합니다...


C:\Users\User\AppData\Local\Temp\ipykernel_4144\3080662519.py:144: UserWarning: Argument(s) 'always_apply' are not valid for transform Resize
  A.Resize(height, width, always_apply=True),



--- 테스트 결과 ---
이미지: C:\Users\User\DBNet_OCR\data\crop\images\rec_crop_00000000.png
예측된 텍스트: '우'


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader # DataLoader는 평가를 위해 필요
from tqdm import tqdm
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- 기본 설정 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_HEIGHT = 32
IMG_WIDTH = 100

# ====================================================================================
# 0. 유틸리티 함수: 문자셋 생성
# ====================================================================================
def generate_character_set(gt_file):
    """gt.txt 파일에서 모든 고유 문자를 추출하여 문자셋을 생성합니다."""
    if not os.path.exists(gt_file):
        print(f"오류: gt.txt 파일을 찾을 수 없습니다! 경로를 확인하세요: {gt_file}")
        return None

    all_characters = set()
    with open(gt_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                _, text = line.strip().split('\t', 1)
                for char in text:
                    all_characters.add(char)
            except ValueError:
                # 탭으로 분리되지 않은 줄은 건너뜁니다.
                continue

    sorted_characters = sorted(list(all_characters))
    final_charset = "".join(sorted_characters)
    
    print("="*50)
    print("CHARACTER_SET 생성이 완료되었습니다.")
    print(f"총 글자 수: {len(final_charset)}")
    print("="*50)
    
    return final_charset

# ====================================================================================
# 1. 레이블 변환기 (CTCLabelConverter)
# ====================================================================================
class CTCLabelConverter:
    """텍스트와 인덱스 간의 변환을 담당하는 클래스"""
    def __init__(self, character_set):
        # 0번 인덱스는 CTC Loss를 위한 'blank' 토큰으로 예약합니다.
        self.character_set = ["-"] + list(character_set)
        
        # 문자를 인덱스로 변환하는 딕셔너리
        self.char_to_idx = {char: i for i, char in enumerate(self.character_set)}
        # 인덱스를 문자로 변환하는 딕셔너리
        self.idx_to_char = {i: char for i, char in enumerate(self.character_set)}
        
        print(f"인식 모델의 클래스 개수 (blank 포함): {self.get_num_classes()}")

    def encode(self, text):
        """입력된 텍스트 문자열을 숫자 인덱스의 리스트로 변환합니다."""
        indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices):
        """모델의 출력(인덱스 시퀀스)을 텍스트 문자열로 디코딩합니다."""
        text = []
        last_idx = 0
        for idx in indices:
            idx_item = idx.item()
            if idx_item == 0:  # blank 토큰은 무시
                last_idx = 0
                continue
            if idx_item == last_idx:  # 연속 중복 문자 무시
                continue
            
            text.append(self.idx_to_char[idx_item])
            last_idx = idx_item
            
        return "".join(text)

    def get_num_classes(self):
        """blank 토큰을 포함한 전체 클래스의 개수를 반환합니다."""
        return len(self.character_set)

# ====================================================================================
# 2. 데이터 파이프라인 (Dataset for Evaluation)
# ====================================================================================
class RecognitionDataset(Dataset):
    """인식용 데이터셋을 위한 클래스"""
    def __init__(self, gt_file_path, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.samples = []
        with open(gt_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    filename, text = line.strip().split('\t', 1) # 탭 기준으로 한 번만 분리
                    self.samples.append((filename, text))
                except ValueError:
                    print(f"경고: 잘못된 형식의 라인 발견 - {line.strip()}")
                    continue

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        filename, text = self.samples[idx]
        image_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(image_path)
        if image is None:
            # 이미지를 읽을 수 없는 경우 빈 이미지와 텍스트를 반환하거나 오류 처리
            # 여기서는 편의상 빈 이미지와 텍스트를 반환하고 경고를 출력합니다.
            print(f"경고: 이미지를 읽을 수 없습니다. 건너뛰기: {image_path}")
            # 대체 이미지 생성 (예: 검은색 이미지)
            image = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8) 
            text = "" # 이 샘플의 텍스트도 비웁니다.
            
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.transform:
                image = self.transform(image=image)['image']
            
        return image, text # 이미지 읽기 실패 시 image는 np.array, 성공 시 torch.Tensor


def get_recognition_transforms(height, width):
    """인식 모델 추론을 위한 데이터 전처리 파이프라인을 정의합니다."""
    return A.Compose([
        A.Resize(height, width, always_apply=True),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2(),
    ])

def recognition_collate_fn_for_inference(batch):
    """
    추론 시 가변 길이 텍스트를 처리하기 위한 커스텀 collate 함수.
    이 함수는 학습 시와 달리 텍스트를 인코딩할 필요 없이 그대로 반환합니다.
    """
    images, texts = zip(*batch)
    
    # 필터링: 유효한 이미지(torch.Tensor)만 필터링합니다.
    valid_images = [img for img in images if isinstance(img, torch.Tensor)]
    valid_texts = [text for i, text in enumerate(texts) if isinstance(images[i], torch.Tensor)]

    if not valid_images:
        return None, None # 유효한 이미지가 없는 경우

    images = torch.stack(valid_images, 0)
    return images, valid_texts


# ====================================================================================
# 3. 텍스트 인식 모델 (CRNN) 아키텍처
# ====================================================================================
class CRNN(nn.Module):
    def __init__(self, num_chars, rnn_hidden_size=256, rnn_layers=2):
        super().__init__()
        
        # --- 1. CNN 특징 추출기 (VGG 스타일) ---
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 64, 16, 50)
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 128, 8, 25)
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 256, 4, 25)
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 512, 2, 25)
            nn.Conv2d(512, 512, (2,1), 1, 0), nn.BatchNorm2d(512), nn.ReLU(True)  # -> (B, 512, 1, 25)
        )
        
        # --- 2. RNN (LSTM) 문맥 학습기 ---
        self.rnn = nn.LSTM(
            input_size=512,             # CNN 출력의 채널 수
            hidden_size=rnn_hidden_size,
            num_layers=rnn_layers,
            bidirectional=True,         # 양방향 RNN으로 더 넓은 문맥 파악
            dropout=0.5
        )
        
        # --- 3. Classifier (분류기) ---
        self.classifier = nn.Linear(rnn_hidden_size * 2, num_chars) # 양방향이므로 *2

    def forward(self, x):
        # 1. CNN을 통과시켜 이미지 특징 추출
        features = self.cnn(x)  # -> (Batch, Channels, Height, Width) = (B, 512, 1, 25)
        
        # 2. RNN 입력 형식으로 변환: (SeqLen, Batch, InputSize)
        b, c, h, w = features.size()
        assert h == 1, "CNN 출력의 높이는 1이어야 합니다."
        features = features.squeeze(2)      # 높이(H) 차원 제거 -> (B, 512, 25)
        features = features.permute(2, 0, 1)  # 차원 순서 변경 -> (W, B, C) = (25, B, 512)
        
        # 3. RNN을 통과시켜 문맥 정보 학습
        rnn_output, _ = self.rnn(features) # -> (SeqLen, Batch, HiddenSize*2)
        
        # 4. 각 시퀀스 스텝에 대해 문자 분류
        output = self.classifier(rnn_output) # -> (SeqLen, Batch, NumClasses)
        return output

# ====================================================================================
# 4. 학습된 모델을 사용한 추론(Inference) 및 평가 모듈
# ====================================================================================
class Recognizer:
    """학습된 CRNN 모델을 사용하여 텍스트 인식을 수행하는 클래스"""
    def __init__(self, model_path, converter, device="cpu", img_height=32, img_width=100):
        self.device = device
        self.converter = converter
        self.img_height = img_height
        self.img_width = img_width

        # 모델 로드 (학습 시 사용한 CRNN 아키텍처와 동일하게 초기화)
        self.model = CRNN(num_chars=self.converter.get_num_classes()).to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval() # 추론 모드로 설정

        # 추론 시 사용할 이미지 전처리 (학습 시와 동일한 정규화)
        self.transform = get_recognition_transforms(self.img_height, self.img_width)
        print(f"Recognizer가 '{model_path}' 모델을 성공적으로 로드했습니다.")

    def predict(self, image_input):
        """
        이미지 파일 경로 또는 이미 전처리된 PyTorch 텐서로부터 텍스트를 예측합니다.
        Args:
            image_input (str or torch.Tensor): 예측할 이미지 파일의 경로 또는 (1, C, H, W) 형태의 PyTorch 텐서.
        Returns:
            str: 예측된 텍스트.
        """
        if isinstance(image_input, str): # 이미지 경로가 주어졌을 경우
            image_path = image_input
            if not os.path.exists(image_path):
                print(f"오류: 이미지를 찾을 수 없습니다: {image_path}")
                return ""

            image = cv2.imread(image_path)
            if image is None:
                print(f"오류: 이미지를 읽을 수 없습니다. 경로 또는 파일 손상 확인: {image_path}")
                return ""

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            processed_image = self.transform(image=image)['image']
            processed_image = processed_image.unsqueeze(0).to(self.device) # 배치 차원 추가
        elif isinstance(image_input, torch.Tensor): # 이미 텐서 형태일 경우
            processed_image = image_input.to(self.device)
            if processed_image.dim() == 3: # (C, H, W) 형태면 배치 차원 추가
                processed_image = processed_image.unsqueeze(0)
        else:
            print("오류: image_input은 이미지 경로(str) 또는 PyTorch 텐서여야 합니다.")
            return ""

        with torch.no_grad():
            output = self.model(processed_image)
            preds_indices = output.argmax(2).permute(1, 0) # (SeqLen, Batch, NumClasses) -> (SeqLen, Batch) -> (Batch, SeqLen)
            
            decoded_text = self.converter.decode(preds_indices[0]) # 첫 번째 (유일한) 배치 샘플
        return decoded_text

def evaluate_recognizer_performance(model_path, converter, gt_file_path, image_dir, device="cpu", batch_size=32):
    """
    학습된 모델의 성능을 평가하는 함수.
    Args:
        model_path (str): 학습된 모델(.pth) 파일 경로.
        converter (CTCLabelConverter): 레이블 변환기 객체.
        gt_file_path (str): Ground Truth 텍스트 파일 경로.
        image_dir (str): 이미지 파일들이 저장된 디렉토리 경로.
        device (str): 모델을 로드할 디바이스 ('cuda' 또는 'cpu').
        batch_size (int): 평가 시 사용할 배치 크기.
    """
    print("\n===== 모델 성능 평가 시작 =====")
    recognizer = Recognizer(model_path, converter, device, IMG_HEIGHT, IMG_WIDTH)

    # 평가 데이터셋 및 DataLoader 준비
    eval_dataset = RecognitionDataset(
        gt_file_path=gt_file_path,
        image_dir=image_dir,
        transform=get_recognition_transforms(IMG_HEIGHT, IMG_WIDTH)
    )

    if len(eval_dataset) == 0:
        print("오류: 평가할 데이터셋에 샘플이 없습니다.")
        return

    eval_loader = DataLoader(
        dataset=eval_dataset,
        batch_size=batch_size,
        shuffle=False, # 평가 시에는 섞을 필요 없음
        num_workers=0, # Windows 환경에서는 0이 안정적
        collate_fn=recognition_collate_fn_for_inference
    )

    total_samples = 0
    correct_predictions = 0

    for images, ground_truth_texts in tqdm(eval_loader, desc="모델 평가 중"):
        if images is None or ground_truth_texts is None: # 이미지 로드 실패 등으로 인해 유효한 배치가 아닌 경우
            continue
            
        # 모델 예측
        with torch.no_grad():
            outputs = recognizer.model(images.to(device))
            preds_indices = outputs.argmax(2).permute(1, 0) # (Batch, SeqLen)

        # 예측 결과 디코딩 및 비교
        for i in range(preds_indices.size(0)):
            predicted_text = recognizer.converter.decode(preds_indices[i])
            ground_truth_text = ground_truth_texts[i] # 해당 배치 샘플의 실제 텍스트

            total_samples += 1
            if predicted_text == ground_truth_text:
                correct_predictions += 1
            # else: # 틀린 예측을 보고 싶다면 주석 해제
            #     print(f"GT: '{ground_truth_text:<20}' | PRED: '{predicted_text}'")

    accuracy = (correct_predictions / total_samples) * 100 if total_samples > 0 else 0
    print(f"\n총 샘플 수: {total_samples}")
    print(f"정답 수: {correct_predictions}")
    print(f"정확도: {accuracy:.2f}%")
    print("===== 모델 성능 평가 완료 =====")


# ====================================================================================
# 메인 실행 블록
# ====================================================================================
if __name__ == '__main__':
    # --- [1단계: 설정] 사용하려는 모델 파일 및 데이터 경로를 여기에 정의합니다. ---
    # !!! 중요: 여기에 실제 모델 파일 경로와 gt.txt, 이미지 폴더 경로를 넣어주세요 !!!
    # 예시:
    MODEL_PATH = "crnn_recognizer_final.pth" # 저장된 모델 파일 이름
    GT_FILE_PATH = r"C:\Users\User\DBNet_OCR\data\crop\gt.txt" # gt.txt 파일 경로
    IMAGE_DIR = r"C:\Users\User\DBNet_OCR\data\crop\images"    # 이미지 폴더 경로

    # --- [2단계: 문자셋 생성] gt.txt 파일로부터 문자셋을 생성합니다. ---
    print(">> 1. 문자셋(CHARACTER_SET)을 생성합니다...")
    charset = generate_character_set(GT_FILE_PATH)
    
    if charset:
        # --- [3단계: CTCLabelConverter 초기화] ---
        label_converter = CTCLabelConverter(charset)

        # --- [4단계: 모델 로드 및 성능 평가] ---
        # test_recognizer_performance 함수를 호출하여 모델을 로드하고 평가를 시작합니다.
        evaluate_recognizer_performance(MODEL_PATH, label_converter, GT_FILE_PATH, IMAGE_DIR, DEVICE)
    else:
        print("문자셋 생성에 실패하여 모델 평가를 시작할 수 없습니다.")

C:\Users\User\.conda\envs\Tensor210Py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>> 1. 문자셋(CHARACTER_SET)을 생성합니다...
CHARACTER_SET 생성이 완료되었습니다.
총 글자 수: 1550
인식 모델의 클래스 개수 (blank 포함): 1551

===== 모델 성능 평가 시작 =====


C:\Users\User\AppData\Local\Temp\ipykernel_20656\1995971611.py:217: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_locat

Recognizer가 'crnn_recognizer_final.pth' 모델을 성공적으로 로드했습니다.
경고: 잘못된 형식의 라인 발견 - rec_crop_00002547.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00007143.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00012858.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00044170.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00097253.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00123738.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00127828.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00208717.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00249756.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00329351.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00345028.png


모델 평가 중: 100%|██████████████████████████████████████████████████████████████| 12175/12175 [30:22<00:00,  6.68it/s]


총 샘플 수: 389574
정답 수: 383212
정확도: 98.37%
===== 모델 성능 평가 완료 =====


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import jiwer # CER 계산을 위해 필요합니다.

# --- 기본 설정 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_HEIGHT = 32
IMG_WIDTH = 100

# ====================================================================================
# 0. 유틸리티 함수: 문자셋 생성
# ====================================================================================
def generate_character_set(gt_file):
    """gt.txt 파일에서 모든 고유 문자를 추출하여 문자셋을 생성합니다."""
    if not os.path.exists(gt_file):
        print(f"오류: gt.txt 파일을 찾을 수 없습니다! 경로를 확인하세요: {gt_file}")
        return None

    all_characters = set()
    with open(gt_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                # 파일명과 텍스트가 탭으로 구분되어 있다고 가정합니다.
                _, text = line.strip().split('\t', 1) 
                for char in text:
                    all_characters.add(char)
            except ValueError:
                # 탭으로 분리되지 않은 줄은 건너뜁니다.
                continue

    sorted_characters = sorted(list(all_characters))
    final_charset = "".join(sorted_characters)
    
    print("="*50)
    print("CHARACTER_SET 생성이 완료되었습니다.")
    print(f"총 글자 수: {len(final_charset)}")
    print("="*50)
    
    return final_charset

# ====================================================================================
# 1. 레이블 변환기 (CTCLabelConverter)
# ====================================================================================
class CTCLabelConverter:
    """텍스트와 인덱스 간의 변환을 담당하는 클래스"""
    def __init__(self, character_set):
        # 0번 인덱스는 CTC Loss를 위한 'blank' 토큰으로 예약합니다.
        self.character_set = ["-"] + list(character_set)
        
        # 문자를 인덱스로 변환하는 딕셔너리
        self.char_to_idx = {char: i for i, char in enumerate(self.character_set)}
        # 인덱스를 문자로 변환하는 딕셔너리
        self.idx_to_char = {i: char for i, char in enumerate(self.character_set)}
        
        print(f"인식 모델의 클래스 개수 (blank 포함): {self.get_num_classes()}")

    def encode(self, text):
        """입력된 텍스트 문자열을 숫자 인덱스의 리스트로 변환합니다."""
        indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices):
        """모델의 출력(인덱스 시퀀스)을 텍스트 문자열로 디코딩합니다."""
        text = []
        last_idx = 0
        for idx in indices:
            idx_item = idx.item()
            if idx_item == 0:  # blank 토큰은 무시 (CTC Blank)
                last_idx = 0
                continue
            if idx_item == last_idx:  # 연속 중복 문자 무시 (CTC Collapse)
                continue
            
            text.append(self.idx_to_char[idx_item])
            last_idx = idx_item
            
        return "".join(text)

    def get_num_classes(self):
        """blank 토큰을 포함한 전체 클래스의 개수를 반환합니다."""
        return len(self.character_set)

# ====================================================================================
# 2. 데이터 파이프라인 (Dataset for Evaluation)
# ====================================================================================
class RecognitionDataset(Dataset):
    """인식용 데이터셋을 위한 클래스 (평가용)"""
    def __init__(self, gt_file_path, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.samples = []
        with open(gt_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    filename, text = line.strip().split('\t', 1) 
                    self.samples.append((filename, text))
                except ValueError:
                    print(f"경고: 잘못된 형식의 라인 발견 - {line.strip()}")
                    continue

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        filename, text = self.samples[idx]
        image_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(image_path)
        if image is None:
            # 이미지를 읽을 수 없는 경우, 경고 출력 및 대체 이미지/텍스트 반환
            print(f"경고: 이미지를 읽을 수 없습니다. 건너뛰기: {image_path}")
            # 이 경우 해당 샘플은 평가에서 제외될 수 있도록 None을 반환하거나,
            # 특정 값을 반환하여 collate_fn에서 처리하게 할 수 있습니다.
            # 여기서는 편의상 numpy 배열로 된 0 값을 반환하여 이후 필터링합니다.
            return np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8), "" 
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
            
        return image, text 


def get_recognition_transforms(height, width):
    """인식 모델 추론을 위한 데이터 전처리 파이프라인을 정의합니다."""
    return A.Compose([
        A.Resize(height, width, always_apply=True),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2(),
    ])

def recognition_collate_fn_for_inference(batch):
    """
    추론 시 가변 길이 텍스트를 처리하기 위한 커스텀 collate 함수.
    이미지 로드 실패 샘플을 필터링하고, 유효한 배치만 구성합니다.
    """
    images, texts = zip(*batch)
    
    # 이미지 로드 실패 등으로 인해 np.array (0 값)가 반환된 경우를 필터링
    # 유효한 PyTorch 텐서만 모읍니다.
    valid_samples = [(img, text) for img, text in zip(images, texts) if isinstance(img, torch.Tensor)]

    if not valid_samples:
        return None, None # 유효한 이미지가 없는 경우

    valid_images, valid_texts = zip(*valid_samples)
    images_tensor = torch.stack(valid_images, 0)
    return images_tensor, list(valid_texts) # texts는 리스트 형태로 유지

# ====================================================================================
# 3. 텍스트 인식 모델 (CRNN) 아키텍처 (학습 시와 동일해야 합니다)
# ====================================================================================
class CRNN(nn.Module):
    def __init__(self, num_chars, rnn_hidden_size=256, rnn_layers=2):
        super().__init__()
        
        # --- 1. CNN 특징 추출기 (VGG 스타일) ---
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 64, 16, 50)
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2), # -> (B, 128, 8, 25)
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 256, 4, 25)
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)), # -> (B, 512, 2, 25)
            nn.Conv2d(512, 512, (2,1), 1, 0), nn.BatchNorm2d(512), nn.ReLU(True)  # -> (B, 512, 1, 25)
        )
        
        # --- 2. RNN (LSTM) 문맥 학습기 ---
        self.rnn = nn.LSTM(
            input_size=512,             # CNN 출력의 채널 수
            hidden_size=rnn_hidden_size,
            num_layers=rnn_layers,
            bidirectional=True,         # 양방향 RNN으로 더 넓은 문맥 파악
            dropout=0.5
        )
        
        # --- 3. Classifier (분류기) ---
        self.classifier = nn.Linear(rnn_hidden_size * 2, num_chars) # 양방향이므로 *2

    def forward(self, x):
        # 1. CNN을 통과시켜 이미지 특징 추출
        features = self.cnn(x)  # -> (Batch, Channels, Height, Width) = (B, 512, 1, 25)
        
        # 2. RNN 입력 형식으로 변환: (SeqLen, Batch, InputSize)
        b, c, h, w = features.size()
        assert h == 1, "CNN 출력의 높이는 1이어야 합니다."
        features = features.squeeze(2)      # 높이(H) 차원 제거 -> (B, 512, 25)
        features = features.permute(2, 0, 1)  # 차원 순서 변경 -> (W, B, C) = (25, B, 512)
        
        # 3. RNN을 통과시켜 문맥 정보 학습
        rnn_output, _ = self.rnn(features) # -> (SeqLen, Batch, HiddenSize*2)
        
        # 4. 각 시퀀스 스텝에 대해 문자 분류
        output = self.classifier(rnn_output) # -> (SeqLen, Batch, NumClasses)
        return output

# ====================================================================================
# 4. 학습된 모델 로드 및 성능 평가 스크립트
# ====================================================================================
def main():
    # --- [1단계: 설정] 사용하려는 모델 파일 및 데이터 경로를 여기에 정의합니다. ---
    # !!! 중요: 여기에 실제 모델 파일 경로와 gt.txt, 이미지 폴더 경로를 넣어주세요 !!!
    MODEL_PATH = "crnn_recognizer_final.pth" 
    GT_FILE_PATH = r"C:\Users\User\DBNet_OCR\data\crop\gt.txt" 
    IMAGE_DIR = r"C:\Users\User\DBNet_OCR\data\crop\images"    

    # --- [2단계: 문자셋 생성] gt.txt 파일로부터 문자셋을 생성합니다. ---
    print(">> 1. 문자셋(CHARACTER_SET)을 생성합니다...")
    charset = generate_character_set(GT_FILE_PATH)
    
    if charset is None: # 문자셋 생성 실패 시 종료
        print("문자셋 생성에 실패하여 모델 평가를 시작할 수 없습니다.")
        return

    # --- [3단계: CTCLabelConverter 초기화] ---
    label_converter = CTCLabelConverter(charset)

    # --- [4단계: 모델 로드] ---
    print(f"\n>> 2. 모델 '{MODEL_PATH}'를 로드합니다...")
    try:
        model = CRNN(num_chars=label_converter.get_num_classes()).to(DEVICE)
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        model.eval() # 평가 모드 설정
        print("모델 로드 성공.")
    except FileNotFoundError:
        print(f"오류: 모델 파일 '{MODEL_PATH}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")
        return
    except Exception as e:
        print(f"오류: 모델 로드 중 문제가 발생했습니다: {e}")
        return

    # --- [5단계: 데이터셋 및 DataLoader 준비 (평가용)] ---
    print("\n>> 3. 평가 데이터셋을 준비합니다...")
    eval_dataset = RecognitionDataset(
        gt_file_path=GT_FILE_PATH,
        image_dir=IMAGE_DIR,
        transform=get_recognition_transforms(IMG_HEIGHT, IMG_WIDTH)
    )

    if len(eval_dataset) == 0:
        print("오류: 평가할 데이터셋에 샘플이 없습니다. gt.txt와 이미지 폴더를 확인해주세요.")
        return

    # 배치 사이즈는 평가 시 자유롭게 설정할 수 있습니다.
    EVAL_BATCH_SIZE = 32 
    eval_loader = DataLoader(
        dataset=eval_dataset,
        batch_size=EVAL_BATCH_SIZE,
        shuffle=False, # 평가 시에는 데이터 순서를 섞을 필요가 없습니다.
        num_workers=0, # Windows 환경에서는 0이 안정적입니다.
        collate_fn=recognition_collate_fn_for_inference # 커스텀 collate 함수 사용
    )
    print(f"총 {len(eval_dataset)}개의 샘플을 평가합니다.")

    # --- [6단계: 모델 성능 평가] ---
    print("\n>> 4. 모델 성능 평가를 시작합니다...")
    total_samples = 0
    correct_predictions = 0 # 문자열 완전 일치 기준
    
    all_ground_truths = []
    all_predictions = []

    for images, ground_truth_texts in tqdm(eval_loader, desc="모델 평가 진행 중"):
        if images is None or ground_truth_texts is None:
            # collate_fn에서 필터링된 유효하지 않은 배치 건너뛰기
            continue
            
        with torch.no_grad(): # 그래디언트 계산 비활성화 (메모리 절약 및 속도 향상)
            outputs = model(images.to(DEVICE)) # 이미지 텐서를 모델 입력으로 사용
            # CTC 디코딩을 위해 예측된 확률 분포에서 가장 높은 확률의 인덱스 선택
            # outputs: (SeqLen, Batch, NumClasses) -> argmax(2) -> (SeqLen, Batch)
            # -> permute(1, 0) -> (Batch, SeqLen)
            preds_indices = outputs.argmax(2).permute(1, 0) 

        # 배치 내 각 샘플에 대해 예측 결과 디코딩 및 비교
        for i in range(preds_indices.size(0)):
            predicted_text = label_converter.decode(preds_indices[i])
            ground_truth_text = ground_truth_texts[i] # 해당 배치 샘플의 실제 텍스트

            total_samples += 1
            if predicted_text == ground_truth_text:
                correct_predictions += 1
            
            # CER/WER 계산을 위해 예측과 실제 텍스트 저장
            all_ground_truths.append(ground_truth_text)
            all_predictions.append(predicted_text)

    # --- [7단계: 결과 출력] ---
    accuracy = (correct_predictions / total_samples) * 100 if total_samples > 0 else 0
    
    print("\n===== 모델 성능 평가 결과 =====")
    print(f"총 평가 샘플 수: {total_samples}")
    print(f"문자열 완전 일치 정답 수: {correct_predictions}")
    print(f"정확도 (문자열 완전 일치 기준): {accuracy:.2f}%")

    # CER (Character Error Rate) 계산
    # jiwer 라이브러리를 사용하여 CER을 계산합니다.
    try:
        # jiwer.measures.cer 함수는 리스트의 리스트를 기대할 수 있으므로, 단일 문자열 리스트로 전달
        # jiwer는 내부적으로 단어를 토큰화하므로, OCR에서는 주로 문자 단위의 비교가 중요합니다.
        # 따라서, 문자 단위로 CER을 계산하려면 각 문자열을 공백으로 구분된 문자열로 변환하는 것이 일반적입니다.
        # 예: "hello" -> "h e l l o"
        
        # 문자를 띄어쓰기로 분리하여 CER 계산 (더 정확한 문자 단위 CER)
        processed_ground_truths = [" ".join(list(s)) for s in all_ground_truths]
        processed_predictions = [" ".join(list(s)) for s in all_predictions]

        cer_value = jiwer.cer(processed_ground_truths, processed_predictions)
        print(f"문자 오류율 (Character Error Rate, CER): {cer_value * 100:.2f}%")
    except Exception as e:
        print(f"CER 계산 중 오류 발생: {e}. 'pip install jiwer'를 실행했는지 확인해주세요.")
    
    print("==============================")

# 메인 실행
if __name__ == '__main__':
    main()

>> 1. 문자셋(CHARACTER_SET)을 생성합니다...
CHARACTER_SET 생성이 완료되었습니다.
총 글자 수: 1550
인식 모델의 클래스 개수 (blank 포함): 1551

>> 2. 모델 'crnn_recognizer_final.pth'를 로드합니다...
모델 로드 성공.

>> 3. 평가 데이터셋을 준비합니다...
경고: 잘못된 형식의 라인 발견 - rec_crop_00002547.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00007143.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00012858.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00044170.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00097253.png


C:\Users\User\AppData\Local\Temp\ipykernel_20656\968197438.py:230: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=DE

경고: 잘못된 형식의 라인 발견 - rec_crop_00123738.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00127828.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00208717.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00249756.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00329351.png
경고: 잘못된 형식의 라인 발견 - rec_crop_00345028.png
총 389574개의 샘플을 평가합니다.

>> 4. 모델 성능 평가를 시작합니다...


모델 평가 진행 중: 100%|█████████████████████████████████████████████████████████| 12175/12175 [15:58<00:00, 12.70it/s]



===== 모델 성능 평가 결과 =====
총 평가 샘플 수: 389574
문자열 완전 일치 정답 수: 383212
정확도 (문자열 완전 일치 기준): 98.37%
문자 오류율 (Character Error Rate, CER): 1.77%
